In [68]:
import internal_ref as ref
import numpy as np
import pandas as pd
import time
import os.path

from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    LinearColorMapper,
    ColorBar,
    LogTicker,
    BasicTicker
)
from bokeh.palettes import Viridis6, magma, viridis
from bokeh.plotting import figure, show, output_file

from bokeh.sampledata.us_counties import data as counties

from bokeh.io import show, export_png




# Choropleths - Health Insurance Ratio 2008-2014

In [26]:
hi = ref.Home + '/HI.csv'
df_hi = pd.read_csv(hi)

In [27]:
df_hi['HI_ratio'] = df_hi['Est_1_HI_pop']/df_hi['Est_total_pop']

df_hi['county_id'] = df_hi['Id2'].map(lambda x: (int(str(x)[:-3]), int(str(x)[-3:])))

In [28]:
df_hi1= df_hi.drop(['Id', 'Id2', 'Geography', 'Est_total_pop', 'Est_1_HI_pop',
       'Est_0_HI_pop'], axis=1)

In [29]:
df_hi1 = df_hi1[['county_id', 'HI_ratio', 'Year']]

In [30]:
hi_master = {}

def hi_master_dct_maker(year):
    df_tempyear = df_hi1[df_hi1.Year == year]
    df_tempyear.drop(['Year'], axis=1, inplace=True)
    df_tempyear.set_index(['county_id'], inplace=True)
    dct_tempyear = df_tempyear.to_dict()['HI_ratio']
    
    hi_master[year] = dct_tempyear

for year in range(2008, 2015):
    hi_master_dct_maker(year)    

/Users/allandong/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [184]:
hi_range = np.sort(df_hi1.HI_ratio.unique())

In [29]:
def my_range(start, end, step):
    while start <= end:
        yield start
        start += step

In [217]:

palette = viridis(256)
#palette.reverse()
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")


for year in my_range(2008, 2014, 1):



    counties = {
        code: county for code, county in counties.items()
    }

    county_xs = [county["lons"] for county in counties.values() if county["state"] not in EXCLUDED]
    county_ys = [county["lats"] for county in counties.values() if county["state"] not in EXCLUDED]

    county_names = [county['name'] for county in counties.values()]
    county_rates = [hi_master[year][county_id] for county_id in hi_master[year]]
    color_mapper = LinearColorMapper(palette=palette, low = hi_range[0], high=hi_range[-1])

    source = ColumnDataSource(data=dict(
        x=county_xs,
        y=county_ys,
        name=county_names,
        rate=county_rates,
    ))

#    TOOLS = "pan,wheel_zoom,reset,hover,save"

    figure_titlename = "Health Insurance Rate, %s" % year
    p = figure(
        title = figure_titlename, 
        #tools=TOOLS,
        plot_width=1000, 
        plot_height=700,
        x_axis_location=None, 
        y_axis_location=None
    )
    p.grid.grid_line_color = None

    p.patches('x', 'y', source=source,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

    p.add_layout(color_bar, 'left')
    
    
#     hover = p.select_one(HoverTool)
#     hover.point_policy = "follow_mouse"
#     hover.tooltips = [
#         ("Name", "@name"),
#         ("Health Insurance Rate)", "@rate%"),
#         ("(Long, Lat)", "($x, $y)"),
#     ]

#    os.chdir(ref.Home)
#     filename ="choropleth_HI_ratio_%s.html" % year
#     titlename ='Choropleth_HI_ratio_%s' % year  
#     output_file(filename, title=titlename)
#     show(p)
#    time.sleep(60)

    os.chdir(ref.Home + 'HI_ratio')
    pngname = "choropleth_HI_ratio_%s.png" % year
    export_png(p, pngname)
#    time.sleep(60)


/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 790), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 792), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 807), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarni

# Choropleth - Death/Birth Ratio 2003-2014

In [63]:
df_deaths[df_deaths.County.str.find('FL')!=-1][df_deaths.Year ==2011]

/Users/allandong/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,County,County_Code,Deaths,Births,Death_Rate,Year,county_id,Death_Birth_Ratio
2292,"Brevard County, FL",12009,36.0,5177.0,6.95,2011,"(12, 9)",6.953834
2293,"Broward County, FL",12011,132.0,21088.0,6.26,2011,"(12, 11)",6.259484
2294,"Collier County, FL",12021,16.0,3202.0,5.00 (Unreliable),2011,"(12, 21)",4.996877
2295,"Duval County, FL",12031,92.0,12416.0,7.41,2011,"(12, 31)",7.409794
2296,"Escambia County, FL",12033,28.0,3838.0,7.30,2011,"(12, 33)",7.295466
2297,"Hillsborough County, FL",12057,146.0,16464.0,8.87,2011,"(12, 57)",8.867833
2298,"Lee County, FL",12071,42.0,6275.0,6.69,2011,"(12, 71)",6.693227
2299,"Manatee County, FL",12081,29.0,3367.0,8.61,2011,"(12, 81)",8.613009
2300,"Marion County, FL",12083,24.0,3353.0,7.16,2011,"(12, 83)",7.157769
2301,"Miami-Dade County, FL",12086,152.0,31366.0,4.85,2011,"(12, 86)",4.846012


In [44]:
deaths = ref.Home + '/deaths.csv'
df_deaths = pd.read_csv(deaths, dtype={'County Code' : int})

In [45]:
deathsColumns = ['County', 'County_Code', 'Deaths', 'Births',
       'Death_Rate', 'Year']
df_deaths.columns = deathsColumns

In [46]:
df_deaths['county_id'] = df_deaths['County_Code'].map(lambda x: (int(str(x)[:-3]), int(str(x)[-3:])))
df_deaths['Death_Birth_Ratio'] = (df_deaths['Deaths']/df_deaths['Births']) * 1000


In [55]:
def remove999(tup):
    if tup[1] == 999:
        return False
    else:
        return True
    
df_deaths1 = df_deaths[df_deaths.county_id.map(remove999) == True]

In [72]:
deaths_master

{2003: {(1, 73): 10.498397612995911,
  (1, 89): 7.8673784771003081,
  (1, 97): 9.3984962406015029,
  (2, 20): 4.655493482309125,
  (4, 13): 6.2965470548408939,
  (4, 19): 6.395756961235473,
  (5, 119): 7.8057241977450129,
  (6, 1): 4.4949026876737728,
  (6, 13): 3.4057367743888594,
  (6, 19): 6.752808259203948,
  (6, 29): 5.6624263108904751,
  (6, 37): 5.2477751141177622,
  (6, 53): 5.3872053872053876,
  (6, 59): 4.4516925246826515,
  (6, 65): 6.4911905271417361,
  (6, 67): 5.0413587195927763,
  (6, 71): 6.7793311492425961,
  (6, 73): 4.5166123204371198,
  (6, 75): 2.8868360277136258,
  (6, 77): 6.7910090865614539,
  (6, 81): 3.8295365278868814,
  (6, 83): 3.2752973625237027,
  (6, 85): 4.3328519053438503,
  (6, 87): 4.6323103647944412,
  (6, 95): 4.4673539518900345,
  (6, 97): 4.2771599657827206,
  (6, 99): 6.6076549058720859,
  (6, 107): 5.9202736482041836,
  (6, 111): 5.9117402164862618,
  (8, 1): 6.1993323795898903,
  (8, 5): 5.3549190535491906,
  (8, 13): 4.4028618602091356,
  (8,

In [57]:
df_deaths2 = df_deaths1.groupby(['Year', 'county_id']).agg({'Death_Birth_Ratio':'first'})


In [25]:
death_ratio_range = np.sort(df_deaths2.Death_Birth_Ratio.unique())

In [26]:
death_ratio_range

array([  1.76140913,   1.76211454,   1.81323663, ...,  15.25553013,
        15.6128025 ,  16.59959759])

In [61]:
deaths_master = {}

def deaths_master_dct_maker(year):
    df_tempyear = df_deaths2.loc[year]
    dct_tempyear = df_tempyear.to_dict()['Death_Birth_Ratio']
    deaths_master[year] = dct_tempyear
    
for year in range(2003,2015):
    deaths_master_dct_maker(year)

In [70]:
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")

palette = magma(256)
palette.reverse()


for year in my_range(2003, 2014, 1):


    counties = {
        code: county for code, county in counties.items()
    }

    county_xs = [county["lons"] for county in counties.values() if county["state"] not in EXCLUDED]
    county_ys = [county["lats"] for county in counties.values() if county["state"] not in EXCLUDED]

    county_names = [county['name'] for county in counties.values()]
    county_rates = [deaths_master[year][county_id] for county_id in deaths_master[year]]
    color_mapper = LinearColorMapper(palette=palette, low = death_ratio_range[0], high=death_ratio_range[-1])

    source = ColumnDataSource(data=dict(
        x=county_xs,
        y=county_ys,
        name=county_names,
        rate=county_rates,
    ))

    TOOLS = "pan,wheel_zoom,reset,hover,save"

    figure_titlename = "Death/Birth Ratio, %s" % year
    p = figure(
        title = figure_titlename,
        tools=TOOLS,
        plot_width=1000,
        plot_height=700,
        x_axis_location=None,
        y_axis_location=None
    )
    p.grid.grid_line_color = None

    p.patches('x', 'y', source=source,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

    p.add_layout(color_bar, 'left')
    
#     hover = p.select_one(HoverTool)
#     hover.point_policy = "follow_mouse"
#     hover.tooltips = [
#         ("Name", "@name"),
#         ("Death/Birth Ratio)", "@rate%"),
#         ("(Long, Lat)", "($x, $y)"),
#     ]

#     os.chdir(ref.Home + '/Death_Birth_ratio')
#     filename ="choropleth_Death_Birth_ratio_%s.html" % year
#     titlename ='Choropleth_Death_Birth_ratio_%s' % year
#     output_file(filename, title=titlename)
#     show(p)
#     time.sleep(60)

    os.chdir(ref.Home + '/Death_Birth_ratio')
    pngname = "choropleth_Death_Birth_ratio_%s.png" % year
    export_png(p, pngname)


/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 229), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 227), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))


# Choropleth - Death/Birth Delta 2003-2014

In [200]:
df_deaths3 = df_deaths2.reset_index()
df_deaths3 = df_deaths3.sort_values(['county_id', 'Year'])
df_deaths3['Delta'] = (df_deaths3.groupby(['county_id']).Death_Birth_Ratio.pct_change())*10
df_deaths3.drop(['Death_Birth_Ratio'], axis=1, inplace=True)

df_deaths4 = df_deaths3.groupby(['Year', 'county_id']).agg({'Delta':'first'})
df_deaths4.fillna(0, inplace=True)

In [201]:
deaths4_master = {}

def deaths4_master_dct_maker(year):
    df_tempyear = df_deaths4.loc[year]
    dct_tempyear = df_tempyear.to_dict()['Delta']
    deaths4_master[year] = dct_tempyear
    
for year in range(2003, 2015):
    deaths4_master_dct_maker(year)

In [202]:
df_deaths4.Delta.unique().mean()

0.14178813302429347

In [205]:
death_delta_range = np.sort(df_deaths4.Delta.unique())

In [218]:
EXCLUDED = ("ak", "hi", "pr", "gu", "vi", "mp", "as")

palette = magma(256)
palette.reverse()

for year in my_range(2003, 2014, 1):




    counties = {
        code: county for code, county in counties.items()
    }

    county_xs = [county["lons"] for county in counties.values() if county["state"] not in EXCLUDED]
    county_ys = [county["lats"] for county in counties.values() if county["state"] not in EXCLUDED]

    county_names = [county['name'] for county in counties.values()]
    county_rates = [deaths4_master[year][county_id] for county_id in deaths4_master[year]]
    color_mapper = LinearColorMapper(palette=palette, low=death_delta_range[0], high=death_delta_range[-1])

    source = ColumnDataSource(data=dict(
        x=county_xs,
        y=county_ys,
        name=county_names,
        rate=county_rates,
    ))

    TOOLS = "pan,wheel_zoom,reset,hover,save"

    figure_titlename = "Death/Birth Delta, %s" % year
    p = figure(
        title = figure_titlename,
        #tools=TOOLS,
        plot_width=1000, 
        plot_height=700,
        x_axis_location=None,
        y_axis_location=None
    )
    p.grid.grid_line_color = None

    p.patches('x', 'y', source=source,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.7, line_color="white", line_width=0.5)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

    p.add_layout(color_bar, 'left')
#     hover = p.select_one(HoverTool)
#     hover.point_policy = "follow_mouse"
#     hover.tooltips = [
#         ("Name", "@name"),
#         ("Death/Birth Delta)", "@rate%"),
#         ("(Long, Lat)", "($x, $y)"),
#     ]
    
#     filename ="choropleth_Death_Birth_Delta_%s.html" % year
#     titlename ='Choropleth_Death_Birth_Delta_%s' % year  
#     output_file(filename, title=titlename)
#     show(p)


    os.chdir(ref.Home + '/Death_Birth_Delta')
    pngname = "choropleth_Death_Birth_Delta_%s.png" % year
    export_png(p, pngname)
    
#    time.sleep(30)

/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 231), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 230), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarning))
/Users/allandong/anaconda/lib/python3.6/site-packages/bokeh/models/sources.py:91: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('name', 3233), ('rate', 231), ('x', 3109), ('y', 3109)
  "Current lengths: %s" % ", ".join(sorted(str((k, len(v))) for k, v in data.items())), BokehUserWarni